In [14]:
import csv
import re


In [15]:
def isInteger(str):
	try:
		num = int(str)
	except ValueError:
		return False
	return True


In [16]:
def intValueOf(str):
	if isInteger(str):
		return int(str)
	else:
		return 0


In [17]:
def percentage(big, small):
	if big != 0 and small != 0:
		return small / big * 100
	else:
		return "NA"


In [18]:
with open('../data/HealthCare.csv', 'r') as healthCareCSV:
    healthCareList = list(healthCareCSV)
    headingRow = True
    reducedCSVList = []
    reducedCSVList.append(['State', 'District', 'Total number of Health Care Centres'])
    for row in healthCareList:
        if headingRow:
            headingRow = False
            continue
        rowList = row.split(",")
        
        noOfSubCentres = intValueOf(rowList[2].strip())
        noOfPrimaryCentres = intValueOf(rowList[3].strip())
        noOfCommunityCentres = intValueOf(rowList[4].strip())
       	noOfSubDivisional = intValueOf(rowList[5].strip())
       	noOfDistrict = intValueOf(rowList[6].strip())
       	totalHealthCareCentres = noOfDistrict + noOfSubDivisional + noOfCommunityCentres + noOfPrimaryCentres + noOfSubCentres
       	# removing special characters
       	stateClean = re.sub('\W+',' ', rowList[0])
       	districtClean = re.sub('\W+',' ', rowList[1])
       	reducedCSVList.append([stateClean, districtClean, totalHealthCareCentres])

    with open('../data/healthCleanReduced.csv', 'w') as healthCleanReduced:
    	writer = csv.writer(healthCleanReduced)
    	writer.writerows(reducedCSVList)


In [19]:
with open('../data/healthCleanReduced.csv', 'r') as healthCareReducedCSV:
	healthCareReducedList = list(healthCareReducedCSV)
	with open('../data/habitation_reduced.csv', 'r') as habitationReduced:
		habitationReducedList = list(habitationReduced)
		stateSetInHabitation = dict()
		headingRow = True
		indexInHabitation = 1
		for rowHabitation in habitationReducedList:
			if headingRow:
				headingRow = False
				continue
			rowList = rowHabitation.split(",")
			stateSetInHabitation[rowList[1].upper().strip()] = indexInHabitation
			indexInHabitation += 1
		#print (stateSetInHabitation)

		stateSetInHealth = dict([])
		headingRow = True
		for rowHealth in healthCareReducedList:
			if headingRow:
				headingRow = False
				continue
			rowList = rowHealth.split(",")
			stateSetInHealth[rowList[1].upper().strip()] = rowList[2]

		commonCount = 0
		uncommonCount = 0

		reducedCSVList = []
		reducedCSVList.append(['State', 'District', 'SC Current', 'ST Current', 'General Current', 'SC Covered', 'ST Covered', 'General Covered', 'Total Health Centres'])

		for stateHabitat in stateSetInHabitation:
			if(stateHabitat in stateSetInHealth):
				commonCount += 1
				tempList = habitationReducedList[stateSetInHabitation[stateHabitat]].split(",")
				reducedCSVList.append([tempList[0], tempList[1], tempList[2], tempList[3], tempList[4], tempList[5], tempList[6], tempList[7].strip(), stateSetInHealth[stateHabitat].strip()])
			else:
				#print (stateHealth)
				uncommonCount += 1
		#print(commonCount)
		#print(uncommonCount)
		#print (reducedCSVList)

		with open('../data/healthAndHabitatMerged.csv', 'w') as merged:
			writer = csv.writer(merged)
			writer.writerows(reducedCSVList)


In [20]:
with open('../data/healthAndHabitatMerged.csv', 'r') as merged:
	mergedList = list(merged)
	headingRow = True

	addedCSVList = []
	addedCSVList.append(['State', 'District', 'SC Current', 'ST Current', 'General Current', 'SC Covered', 'ST Covered', 'General Covered', 'Total Health Centres', 'Total Population', 'People per Health Centre', 'Percentage SC', 'Percentage SC covered',  'Percentage ST', 'Percentage ST covered',  'Percentage General', 'Percentage General Covered'])

	for row in mergedList:
		if headingRow:
			headingRow = False
			continue
		rowList = row.split(",")

		stCurrent = intValueOf(rowList[3])
		scCurrent = intValueOf(rowList[2])
		generalCurrent = intValueOf(rowList[4])
		
		stCovered = intValueOf(rowList[6])
		scCovered = intValueOf(rowList[5])
		generalCovered = intValueOf(rowList[7])

		perStCovered = percentage(stCurrent, stCovered)
		perScCovered = percentage(scCurrent, stCovered)
		perGenCovered = percentage(generalCurrent, generalCovered)

		totalPopulation = stCurrent + scCurrent + generalCurrent

		perSc = percentage(totalPopulation, scCurrent)
		perSt = percentage(totalPopulation, stCurrent)
		perGen = percentage(totalPopulation, generalCurrent)

		peoplePerHealthCentre = "NA"
		if(intValueOf(rowList[8]) != 0):
			peoplePerHealthCentre = totalPopulation / intValueOf(rowList[8])
        
		addedCSVList.append([rowList[0], rowList[1], rowList[2], rowList[3], rowList[4], rowList[5], rowList[6], rowList[7], rowList[8].strip(), totalPopulation, peoplePerHealthCentre, perSc, perScCovered, perSt, perStCovered, perGen, perGenCovered])

	#print(addedCSVList)

	with open('../data/AddedAttributesMerged.csv', 'w') as merged:
		writer = csv.writer(merged)
		writer.writerows(addedCSVList)
